# Get, prepare, and store data

### Import and functions

In [1]:
!pip install yfinance --upgrade --no-cache-dir
!pip install tensorflow==2.0.0

  Created wheel for yfinance: filename=yfinance-0.1.45-cp36-none-any.whl size=14652 sha256=2e4ee08f6da43bfcc40efc57a779c2e56435bbfcea431a344229c278721b290d
  Stored in directory: /tmp/pip-ephem-wheel-cache-prwptk0n/wheels/0c/d1/df/aa9a7744a4ac353cc9a1f2c3aaea7c1f457fc49de4286f2d88
Successfully built yfinance
     |████████████████████████████████| 86.3MB 102kB/s 
     |████████████████████████████████| 3.8MB 28.9MB/s 
     |████████████████████████████████| 450kB 56.3MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
from pandas_datareader import data
import pandas as pd
import yfinance as yf
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import sys
from datetime import datetime
from google.cloud import storage
from google.colab import auth
yf.pdr_override()

In [0]:
auth.authenticate_user()

In [0]:
def _floatlist_feature(value):
    """Returns a float_list from a list of floats."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _float_feature(value):
    """Returns a float_list from a single float."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int_feature(value):
    """Returns a int_list from a single int."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client(project = 'Adversarial-Finance')
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    
def scale_by_company(df, suffix, scale_func):
    comp_df = df.unstack().unstack(level=0).reset_index(level=0, drop=False).rename(columns={'level_0':'company'}).reset_index()
    company_stats = comp_df.groupby('company').agg(['mean','std'])
    for feat, company in df.columns:
        company_mean, company_std = company_stats.loc[company, feat]
        df.loc[:,(f"{feat}_scaled_{suffix}", company)] = scale_func(df.loc[:, (feat, company)],
                                                                    company_mean, company_std)
    return df.drop(columns = [c for c in df.columns if 'scaled' not in c[0]])

def scale_by_year(df, suffix, scale_func):
    year_df = df.unstack().unstack(level=0).reset_index(level=0, drop=False).drop(columns=['level_0'])
    yr_ix = year_df.index.to_period('Y')
    yearly_stats = year_df.groupby(yr_ix).agg(['mean','std'])
    yearly_stats.index = yearly_stats.index.astype(int) + 1970

    df['year'] = df.index.year

    for feat, company in df.columns:

        if feat in ['year', 'mean', 'std']:
            continue

        df['mean'] = df.year.map(yearly_stats.loc[:,(feat, 'mean')])
        df['std'] = df.year.map(yearly_stats.loc[:,(feat, 'std')])
        df.loc[:,(f"{feat}_scaled_{suffix}", company)] = scale_func(df.loc[:, (feat, company)],
                                                                    df['mean'], df['std'])
    return df.drop(columns = [c for c in df.columns if 'scaled' not in c[0]])

def scale_by_both(df, suffix, scale_func):
    yr_ix = df.index.to_period('Y')
    yearly_stats = df.groupby(yr_ix).agg(['mean','std'])
    yearly_stats.index = yearly_stats.index.astype(int) + 1970

    df['year'] = df.index.year

    for feat, company in df.columns:

        if feat in ['year', 'mean', 'std']:
            continue

        df['mean'] = df.year.map(yearly_stats.loc[:,(feat, company, 'mean')])
        df['std'] = df.year.map(yearly_stats.loc[:,(feat, company, 'std')])
        df.loc[:,(f"{feat}_scaled_{suffix}", company)] = scale_func(df.loc[:, (feat, company)],
                                                                    df['mean'], df['std'])
    return df.drop(columns = [c for c in df.columns if 'scaled' not in c[0]])

def z_scale(values, m, s):
    epsilon = sys.float_info.epsilon
    return (values - m)/(s+epsilon)    

#   recommended time series scaling for stocks through https://pdfs.semanticscholar.org/f412/4953553981e32c39273bb2745a140311d160.pdf
# https://arxiv.org/pdf/1812.05519.pdf

def tanh_scale(values, m, s):
    epsilon = sys.float_info.epsilon
    return 0.5 * (np.tanh(0.01 * ((values - m) / (s + epsilon))) + 1)

def write_records(df, target, is_train, num_steps = 60, num_comps = len(stock_ids)):

    CASES_PER_RECORD = 6000
    source_file_name = "temp.tfrecord"
    
    assert df.shape[0] == target.shape[0]
    
    if is_train:
        bucket_name = "adversarial-finance-resources"
        destination_blob_name = "data/training/train_data_{}.tfrecord"
    else:
        bucket_name = "adversarial-finance-resources"
        destination_blob_name = "data/testing/test_data_{}.tfrecord"
    
    cols = df.columns.get_level_values(level = 0).unique()
    examples_written = 0
    records_written = 0
    df['month'] = df.index.month.astype(int)
    df['day'] = df.index.day.astype(int)

    tfwriter =  tf.io.TFRecordWriter("temp.tfrecord")
    
    for i in tqdm(range(df.shape[0] - num_steps)):

        features = {}
        for feat in cols:
            flat_feat_series = df.iloc[i:(i+num_steps)][feat].values.flatten()
            features[feat] = _floatlist_feature(flat_feat_series)

        features.update({'month':_int_feature(df.iloc[(i+num_steps)].month.values.astype(int)[0]),
                     'day':_int_feature(df.iloc[(i+num_steps)].day.values.astype(int)[0]),
                     'scaled_adj_close':_floatlist_feature(target.iloc[(i+num_steps)].values)})

        example = tf.train.Example(features=tf.train.Features(feature=features))
        tfwriter.write(example.SerializeToString())

        examples_written += 1

#         upload every interval and restart
        if examples_written >= CASES_PER_RECORD:
            tfwriter.close()
            upload_blob(bucket_name,
                        source_file_name,
                        destination_blob_name.format(records_written))
            
            tfwriter = tf.python_io.TFRecordWriter("temp.tfrecord")
            records_written += 1
            examples_written = 0

    # upload remainded
    if examples_written > 0:
        tfwriter.close()
        upload_blob(bucket_name,
                    source_file_name,
                    destination_blob_name.format(records_written))

### Download and clean nulls

In [36]:
# stocks:   Apple, AMD, Amazon,, Cisco, IBM, Intel, Microsoft, Nvidia'
stock_ids = ['AAPL', 'AMD', 'AMZN', 'CSCO', 'IBM', 'INTC', 'MSFT', 'NVDA']
df = data.get_data_yahoo(stock_ids,  datetime(1999,9,12))
df.rename(columns={'Adj Close':"Adj_close"}, inplace = True)
df.drop(columns=['Close'], inplace = True)

[*********************100%***********************]  8 of 8 downloaded


In [37]:
# day after September 11th is null.
print(df[df.isnull().any(axis=1)])
# drop row
df.drop(index = df[df.isnull().any(axis=1)].index, inplace = True)
df.isnull().sum().sum()

           Adj_close                         ... Volume                        
                AAPL AMD AMZN CSCO IBM INTC  ...   AMZN CSCO IBM INTC MSFT NVDA
Date                                         ...                               
2001-09-12       NaN NaN  NaN  NaN NaN  NaN  ...    NaN  NaN NaN  NaN  NaN  NaN

[1 rows x 40 columns]


0

### Split data and scale

In [0]:
train_df = df.loc[:"2018-10-30"].copy()
train_target = train_df.pop('Adj_close')
train_target.columns = pd.MultiIndex.from_product([['target'], train_target.columns])
test_df = df.loc["2018-10-31":].copy()
test_target = test_df.pop('Adj_close')
test_target.columns = pd.MultiIndex.from_product([['target'], test_target.columns])

In [0]:
scaled_train_df = pd.concat([scale_by_year(train_df.copy(), 'year', tanh_scale),
                scale_by_both(train_df.copy(), 'company_year', tanh_scale),
                scale_by_company(train_df.copy(), 'company', tanh_scale)], axis = 1)
scaled_test_df = pd.concat([scale_by_year(test_df.copy(), 'year', tanh_scale),
                scale_by_both(test_df.copy(), 'company_year', tanh_scale),
                scale_by_company(test_df.copy(), 'company', tanh_scale)], axis = 1)
scaled_train_target = scale_by_both(train_target.copy(), 'company_year', tanh_scale)
scaled_test_target = scale_by_both(test_target.copy(), 'company_year', tanh_scale)

### Convert to TF-Records and upload to cloud

In [41]:
write_records(scaled_train_df, scaled_train_target, is_train = True)
write_records(scaled_test_df, scaled_test_target, is_train = False)

100%|██████████| 187/187 [00:06<00:00, 29.96it/s]


In [0]:
fname = 'raw_data.csv'
df.to_csv(fname)
upload_blob("adversarial-finance-resources", fname, fname)
dfs = ['scaled_train_df',
        'scaled_train_target',
        'scaled_test_df',
        'scaled_test_target']

for sub_df in dfs:
    fname = sub_df + '.csv'
    eval(sub_df).to_csv(fname)
    upload_blob("adversarial-finance-resources", fname, fname)